---
###  crawling server test 



In [1]:
# Import Library Package
## Flask Server
from flask import Flask, request, jsonify

## Basic
import pandas as pd, numpy as np

## Crawling
import requests # 인터넷에서 Data를 가져오기 위한 Library (웹페이지에 접속하고 HTML 코드를 가져오기 위해 사용)
from bs4 import BeautifulSoup # 웹 페이지 내용을 분석하기 위한 Library (가져온 HTML 코드에서 우리가 필요한 정보를 추출하기 위해 사용)

import time # 대기 시간을 추가하기 위한 Library (요청 사이에 랜덤한 시간을 기다리기 위해 사용)
import random # Random한 대기 시간을 만들기 위한 Library
from tqdm import tqdm # Crawling 진행 상황을 체크하기 위한 Module (진행 상황을 시각적으로 보여주기 위해 사용)

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

## Bag of Words (BoW)
import nltk # Natural Language Toolkit (자연어 처리를 위해 사용)
from konlpy.tag import Okt
from collections import Counter # 단어의 빈도를 계산하기 위해 사용

### 1. nltk Data Download
nltk.download('punkt')

### 2. 한국어 불용어 사전
# *************************************************************************
## 한국어 불용어 모음집 불러오기
stopword_list = pd.read_csv("../Data/updated_stopword.txt", header = None)
# *************************************************************************
stopword_list[0] = stopword_list[0].apply(lambda x: x.strip())
stopwords = stopword_list[0].to_numpy()

## Deep Learning
import torch
from transformers import BertModel, BertTokenizer
from sklearn.metrics.pairwise import cosine_similarity


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/forrestdpark/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
## app 에서 받은 키워드 
keyword1 = "자동차"
keyword2 = "반도체"
keyword3 = "IT"

#  


In [9]:
# 1. NAVER News Web Crawling
## 1-1. 언론사별 랭킹뉴스 Crawling 함수 정의
def get_news_links_by_press (url,artilceNum) :
  """
    headers:
    - 나는 bot이 아니고 사람임을 증명하는 부분이다.
    - 사용하지 않을 시 언론사에서 웹크롤링을 막을 수 있으니 주의할 것!
  """
  headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36"
  }
  response = requests.get(url, headers = headers) # url(페이지)에 접속
  soup = BeautifulSoup(response.content, 'html.parser') # HTML 코드를 파싱(분석)하여 soup 객체에 저장
  
  press_data = {}
  # naver ranking news  
  press_sections = soup.select('.rankingnews_box')
  press_list = [
      "중앙일보",
      "연합뉴스",
      "경향신문",
      "조선일보",
      "JTBC",
      "한국일보",
      "한국경제",
      "MBC",
      "YTN",
      "동아일보"                          
    ]
  for press_section in tqdm(press_sections, desc = "언론사별 뉴스 Crawling") : # tqdm : 크롤링 시각화 
    press_name = press_section.select_one('.rankingnews_name').get_text(strip = True)
    # print(press_name)
    if press_name in press_list:
      print(press_name)
      news_links = set()  # 중복 제거를 위한 set 사용
      for item in press_section.select('li a') :
        title = item.get_text(strip = True)
        link = item['href']
        if title and link and "동영상" not in title :  # Title이 존재하고 "동영상"이 포함되지 않은 경우에만 추가
          news_links.add((title, link))
      press_data[press_name] = list(news_links)[:artilceNum]  # 다시 list로 변환 후 상위 5개만 저장
      # *********************************************
      # 각 언론사별 뉴스 Crawling 후 대기 시간 추가
      time.sleep(random.uniform(0.32, 0.8))
      # *********************************************
  return press_data

## 1-2. 언론사별 랭킹뉴스의 Press, Title, Link를 토대로 하는 DataFrame 생성
base_url = 'https://news.naver.com/main/ranking/popularDay.naver'
press_news_data = get_news_links_by_press(base_url,10)


한국일보


경향신문


중앙일보


동아일보


JTBC


...

한국경제


YTN


연합뉴스


MBC


언론사별 뉴스 Crawling: 100%|██████████| 82/82 [00:05<00:00, 15.45it/s]


In [12]:
print(press_news_data)

None


In [3]:
# 1. NAVER News Web Crawling
## 1-1. 언론사별 랭킹뉴스 Crawling 함수 정의
def get_news_links_by_press (url) :
  """
    headers:
    - 나는 bot이 아니고 사람임을 증명하는 부분이다.
    - 사용하지 않을 시 언론사에서 웹크롤링을 막을 수 있으니 주의할 것!
  """
  headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36"
  }
  response = requests.get(url, headers = headers) # url(페이지)에 접속
  soup = BeautifulSoup(response.content, 'html.parser') # HTML 코드를 파싱(분석)하여 soup 객체에 저장
  
  press_data = {}
  # naver ranking news  
  press_sections = soup.select('.rankingnews_box')
  
  for press_section in tqdm(press_sections, desc = "언론사별 뉴스 Crawling") : # tqdm : 크롤링 시각화 
    press_name = press_section.select_one('.rankingnews_name').get_text(strip = True)
    news_links = set()  # 중복 제거를 위한 set 사용
    for item in press_section.select('li a') :
      title = item.get_text(strip = True)
      link = item['href']
      if title and link and "동영상" not in title :  # Title이 존재하고 "동영상"이 포함되지 않은 경우에만 추가
        news_links.add((title, link))
    press_data[press_name] = list(news_links)[:5]  # 다시 list로 변환 후 상위 5개만 저장
    # *********************************************
    # 각 언론사별 뉴스 Crawling 후 대기 시간 추가
    time.sleep(random.uniform(0.5, 1.0))
    # *********************************************
  
  return press_data

## 1-2. 언론사별 랭킹뉴스의 Press, Title, Link를 토대로 하는 DataFrame 생성
base_url = 'https://news.naver.com/main/ranking/popularDay.naver'
press_news_data = get_news_links_by_press(base_url)

news_list = []
for press_name, news_data in press_news_data.items() :
  for title, link in news_data :
    news_list.append([press_name, title, link])
news_df = pd.DataFrame(news_list, columns = ['Press', 'Title', 'Link'])


# 2. 언론사별 뉴스 기사 본문 Crawling
## 2-1. Chrome Browser와 Chrome Driver Version 확인 및 WebDriver 객체 생성
chrome_options = webdriver.ChromeOptions()
# ******************************************************
chrome_options.add_argument('headless') # Run chrome browser in the background
chrome_options.add_argument('window-size = 1920x1080')  # Chrome Browser Window Size
# ******************************************************
driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()), options = chrome_options)

# 2-2. 언론사별 뉴스 기사 본문 Crawling 함수 정의
def get_article_text (url) :
  driver.get(url)
  html = driver.page_source
  article_soup = BeautifulSoup(html, "html.parser")
  content = article_soup.select_one("#contents")
  if content :
    # 공백과 HTML Tag 제거
    text =''.join(content.text.split())
    # *******************************************
    # 요청 후 임의의 시간만큼 대기 (Page Loaded)
    time.sleep(random.uniform(0.5, 0.8))
    # *******************************************
    
    return text

## 2-3. 뉴스 기사 본문 수집
content_pbar = tqdm(news_df['Link'], desc = "뉴스 기사 본문 Crawling 진행 중", unit = "Link")
news_df['content'] = [get_article_text(url) for url in content_pbar]

## 2-4. Browser 종료 (모든 Tab 종료)
driver.quit()


뉴스 기사 본문 Crawling 진행 중:  48%|████▊     | 194/407 [03:03<03:13,  1.10Link/s]

KeyboardInterrupt: 